In [15]:
import numpy as np
import pandas as pd

import tensorflow as tf
import nfp

print(f"tensorflow {tf.__version__}")
print(f"nfp {nfp.__version__}")

tensorflow 2.8.0
nfp 0+unknown


In [16]:
# Load the input data, here YSI (10.1016/j.combustflame.2017.12.005)
# ysi.csv available from https://github.com/pstjohn/YSIs_for_prediction/
ysi = pd.read_csv('ysi.csv')
ysi.head()

,Species,CAS,Type,SMILES,YSI,YSI_err
0,isocetane,4390-04-9,alkanes,CC(CC(C)(C)C)CC(C)(C)CC(C)(C)C,128.0,6.4
1,"1,4-cyclohexanedimethanol divinyl ether",17351-75-6,oxygenates,C=COCC1CCC(COC=C)CC1,130.5,6.5
2,gamma-Undecalactone,104-67-6,esters,CCCCCCCC1CCC(=O)O1,58.2,2.9
3,ethyl decanoate,110-38-3,esters,CCCCCCCCCC(=O)OCC,58.0,2.9
4,butyl decanoate,30673-36-0,esters,CCCCCCCCCC(=O)OCCCC,82.6,4.1


In [17]:
# Split the data into training, validation, and test sets
valid, test, train = np.split(ysi.SMILES.sample(frac=1., random_state=1), [50, 100])
len(train), len(valid), len(test)

(557, 50, 50)

In [18]:
# Define how to featurize the input molecules
from nfp.preprocessing.mol_preprocessor import SmilesPreprocessor
from nfp.preprocessing.features import get_ring_size


def atom_featurizer(atom):
    """ Return an string representing the atom type
    """

    return str((
        atom.GetSymbol(),
        atom.GetIsAromatic(),
        get_ring_size(atom, max_size=6),
        atom.GetDegree(),
        atom.GetTotalNumHs(includeNeighbors=True)
    ))


def bond_featurizer(bond, flipped=False):
    """ Get a similar classification of the bond type.
    Flipped indicates which 'direction' the bond edge is pointing. """
    
    if not flipped:
        atoms = "{}-{}".format(
            *tuple((bond.GetBeginAtom().GetSymbol(),
                    bond.GetEndAtom().GetSymbol())))
    else:
        atoms = "{}-{}".format(
            *tuple((bond.GetEndAtom().GetSymbol(),
                    bond.GetBeginAtom().GetSymbol())))
    
    btype = str(bond.GetBondType())
    ring = 'R{}'.format(get_ring_size(bond, max_size=6)) if bond.IsInRing() else ''
    
    return " ".join([atoms, btype, ring]).strip()


preprocessor = SmilesPreprocessor(atom_features=atom_featurizer, bond_features=bond_featurizer,
                                  explicit_hs=False)

In [19]:
# Initially, the preprocessor has no data on atom types, so we have to loop over the 
# training set once to pre-allocate these mappings
print("before pre-allocating")
print(preprocessor.atom_tokenizer._data)

for smiles in train:
    preprocessor(smiles, train=True)
    
print()
print("after pre-allocating")
print(preprocessor.atom_tokenizer._data)

before pre-allocating
{'unk': 1}

after pre-allocating
{'unk': 1, "('C', False, 0, 1, 2)": 2, "('C', False, 0, 2, 1)": 3, "('C', False, 0, 2, 2)": 4, "('C', False, 0, 3, 1)": 5, "('C', False, 0, 1, 3)": 6, "('O', False, 0, 2, 0)": 7, "('C', False, 0, 3, 0)": 8, "('O', False, 0, 1, 0)": 9, "('C', False, 'max', 3, 1)": 10, "('C', False, 'max', 2, 2)": 11, "('O', False, 0, 1, 1)": 12, "('C', False, 0, 2, 0)": 13, "('N', False, 0, 1, 0)": 14, "('C', True, 'max', 2, 1)": 15, "('C', True, 'max', 3, 0)": 16, "('N', True, 'max', 2, 0)": 17, "('C', True, 5, 3, 0)": 18, "('C', False, 5, 2, 2)": 19, "('N', False, 5, 2, 1)": 20, "('C', False, 0, 4, 0)": 21, "('C', True, 5, 2, 1)": 22, "('O', True, 5, 2, 0)": 23, "('N', False, 0, 1, 1)": 24, "('C', False, 5, 2, 1)": 25, "('C', False, 5, 3, 1)": 26, "('N', True, 5, 2, 1)": 27, "('N', False, 0, 2, 0)": 28, "('N', False, 0, 2, 1)": 29, "('C', False, 'max', 2, 1)": 30, "('O', False, 5, 2, 0)": 31, "('N', False, 0, 1, 2)": 32, "('C', False, 4, 3, 1)": 3

In [20]:
# Main input types for a SMILES-based prediction
smiles = 'CCO'

# Atom types, as integer classes
preprocessor(smiles, train=True)['atom']

array([ 6,  4, 12], dtype=int32)

In [21]:
# Bond types, as integer classes
preprocessor(smiles, train=True)['bond']

array([3, 3, 4, 5], dtype=int32)

In [22]:
# A connectivity array, where row i indicates bond i connects atom j to atom k
preprocessor(smiles, train=True)['connectivity']

array([[0, 1],
       [1, 0],
       [1, 2],
       [2, 1]])

In [23]:
# Construct the tf.data pipeline. There's a lot of specifying data types and
# expected shapes for tensorflow to pre-allocate the necessary arrays. But 
# essentially, this is responsible for calling the input constructor, batching 
# together multiple molecules, and padding the resulting molecules so that all
# molecules in the same batch have the same number of atoms (we pad with zeros,
# hence why the atom and bond types above start with 1 as the unknown class)

train_dataset = tf.data.Dataset.from_generator(
    lambda: ((preprocessor(row.SMILES, train=False), row.YSI)
             for i, row in ysi[ysi.SMILES.isin(train)].iterrows()),
    output_signature=(preprocessor.output_signature, tf.TensorSpec((), dtype=tf.float32)))\
    .cache().shuffle(buffer_size=200)\
    .padded_batch(batch_size=64)\
    .prefetch(tf.data.experimental.AUTOTUNE)


valid_dataset = tf.data.Dataset.from_generator(
    lambda: ((preprocessor(row.SMILES, train=False), row.YSI)
             for i, row in ysi[ysi.SMILES.isin(valid)].iterrows()),
    output_signature=(preprocessor.output_signature, tf.TensorSpec((), dtype=tf.float32)))\
    .cache()\
    .padded_batch(batch_size=64)\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [24]:
inputs, outputs = next(train_dataset.as_numpy_iterator())
inputs['atom']

2022-04-07 19:37:08.950818: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


array([[ 6,  4, 13, ...,  0,  0,  0],
       [ 6,  4,  4, ...,  0,  0,  0],
       [ 6, 16, 16, ...,  0,  0,  0],
       ...,
       [ 2,  3,  7, ...,  0,  0,  0],
       [25, 25, 31, ...,  0,  0,  0],
       [30, 30, 11, ...,  0,  0,  0]], dtype=int32)

In [25]:
## Define the keras model
from tensorflow.keras import layers

# Input layers
atom = layers.Input(shape=[None], dtype=tf.int64, name='atom')
bond = layers.Input(shape=[None], dtype=tf.int64, name='bond')
connectivity = layers.Input(shape=[None, 2], dtype=tf.int64, name='connectivity')

num_features = 8  # Controls the size of the model

# Convert from a single integer defining the atom state to a vector
# of weights associated with that class
atom_state = layers.Embedding(preprocessor.atom_classes, num_features,
                              name='atom_embedding', mask_zero=True)(atom)

# Ditto with the bond state
bond_state = layers.Embedding(preprocessor.bond_classes, num_features,
                              name='bond_embedding', mask_zero=True)(bond)

# Here we use our first nfp layer. This is an attention layer that looks at
# the atom and bond states and reduces them to a single, graph-level vector. 
# mum_heads * units has to be the same dimension as the atom / bond dimension
global_state = nfp.GlobalUpdate(units=8, num_heads=1)([atom_state, bond_state, connectivity])

for _ in range(3):  # Do the message passing
    new_bond_state = nfp.EdgeUpdate()([atom_state, bond_state, connectivity, global_state])
    bond_state = layers.Add()([bond_state, new_bond_state])
    
    new_atom_state = nfp.NodeUpdate()([atom_state, bond_state, connectivity, global_state])
    atom_state = layers.Add()([atom_state, new_atom_state])
    
    new_global_state = nfp.GlobalUpdate(units=8, num_heads=1)(
        [atom_state, bond_state, connectivity, global_state]) 
    global_state = layers.Add()([global_state, new_global_state])

    
# Since the final prediction is a single, molecule-level property (YSI), we 
# reduce the last global state to a single prediction.
ysi_prediction = layers.Dense(1)(global_state)

# Construct the tf.keras model
model = tf.keras.Model([atom, bond, connectivity], [ysi_prediction])

In [26]:
model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1E-3))

# Fit the model. The first epoch is slower, since it needs to cache
# the preprocessed molecule inputs
model.fit(train_dataset, validation_data=valid_dataset, epochs=25)

Epoch 1/25
9/9 [==============================] - 16s 471ms/step - loss: 145.7803 - val_loss: 132.1886
Epoch 2/25
9/9 [==============================] - 0s 34ms/step - loss: 142.8636 - val_loss: 125.1585
Epoch 3/25
9/9 [==============================] - 0s 41ms/step - loss: 130.8008 - val_loss: 104.9459
Epoch 4/25
9/9 [==============================] - 0s 32ms/step - loss: 118.7174 - val_loss: 99.8784
Epoch 5/25
9/9 [==============================] - 0s 28ms/step - loss: 110.6385 - val_loss: 96.5609
Epoch 6/25
9/9 [==============================] - 0s 30ms/step - loss: 107.1249 - val_loss: 92.6281
Epoch 7/25
9/9 [==============================] - 0s 39ms/step - loss: 101.7515 - val_loss: 82.9378
Epoch 8/25
9/9 [==============================] - 0s 33ms/step - loss: 89.9552 - val_loss: 64.1952
Epoch 9/25
9/9 [==============================] - 0s 30ms/step - loss: 77.1562 - val_loss: 58.7141
Epoch 10/25
9/9 [==============================] - 0s 30ms/step - loss: 68.2843 - val_loss: 55.87

In [27]:
# Here, we create a test dataset that doesn't assume we know the values for the YSI

test_dataset = tf.data.Dataset.from_generator(
    lambda: (preprocessor(smiles, train=False)
             for smiles in test),
    output_signature=preprocessor.output_signature)\
    .padded_batch(batch_size=64)\
    .prefetch(tf.data.experimental.AUTOTUNE)

In [29]:
# Here are the predictions on the test set
test_predictions = model.predict(test_dataset)
test_db_values = ysi.set_index('SMILES').reindex(test).YSI.values

MAE = np.abs(test_db_values - test_predictions.flatten()).mean()
MAE

43.1751065296936

In [31]:
# RMSE
from sklearn.metrics import mean_squared_error
RMSE = mean_squared_error(test_db_values, test_predictions.flatten(), squared=False)
RMSE

91.23847106878615

In [32]:
# “reduce values” = RMSE/std dev of target data
reduce_values = RMSE / (ysi['YSI'].std())
reduce_values

0.38409635473237114

In [30]:
# MAE/MAD
MAE / (ysi['YSI'].mad())

0.2776028795278609